In [1]:
%pip install earthengine-api pycrs numpy

Note: you may need to restart the kernel to use updated packages.


In [2]:
import ee
import geemap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
ee.Initialize(project='ee-lc-trends')

In [3]:
aoi = ee.FeatureCollection('FAO/GAUL/2015/level0').filter(ee.Filter.eq('ADM0_NAME', 'Afghanistan'))

def clipping(image):
  return image.clip(aoi)

def scale(image):
  return image.multiply(2.75e-05).add(-0.2)

img_col = (
    ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
    .filterDate('2000-01-01','2000-12-31')
    .filterBounds(aoi)
    .filter(ee.Filter.lt('CLOUD_COVER', 20))
    .map(scale)
    .map(clipping)
)

img = img_col.reduce(ee.Reducer.percentile([20]))
img = img.select(['SR_B1_p20', 'SR_B2_p20', 'SR_B3_p20', 'SR_B4_p20', 'SR_B5_p20'])

print("Filtered Image")
img

Filtered Image


In [4]:
Map = geemap.Map(center=[33.97, 66.46], zoom=5)
visRGB = {
    'min': -0.2,
    'max': 1,
    'bands': ['SR_B3_p20', 'SR_B2_p20', 'SR_B1_p20']
}
Map.addLayer(img, visRGB, "Image")

In [5]:
def getimage(startY, endY, startM, endM):
    img_col = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2').filterBounds(aoi)
    img_col = img_col.filter(ee.Filter.calendarRange(startY, endY, 'year')).filter(ee.Filter.calendarRange(startM, endM, 'month'))
    img_col = img_col.filter(ee.Filter.lt('CLOUD_COVER', 20))
    
    img = img_col.reduce(ee.Reducer.percentile([20]))
    img = img.clip(aoi)
    img = img.multiply(2.75e-05).add(-0.2)

    img_select = img.select(['SR_B1_p20', 'SR_B2_p20', 'SR_B3_p20', 'SR_B4_p20', 'SR_B5_p20'])
    return img_select

imageTier1 = getimage(1991, 1995, 1, 2)
imageTier2 = getimage(1991, 1995, 3, 4)
imageTier3 = getimage(1991, 1995, 5, 6)
imageTier4 = getimage(1991, 1995, 7, 8)
imageTier5 = getimage(1991, 1995, 9, 10)
imageTier6 = getimage(1991, 1995, 11, 12)

In [6]:
imageTier6

In [8]:
Map = geemap.Map(center=[33.97, 66.46], zoom=5)
Map.addLayer(imageTier1, visRGB, "Image1")
Map.addLayer(imageTier2, visRGB, "Image2")
Map.addLayer(imageTier3, visRGB, "Image3")
Map.addLayer(imageTier4, visRGB, "Image4")
Map.addLayer(imageTier5, visRGB, "Image5")
Map.addLayer(imageTier6, visRGB, "Image6")
Map

Map(center=[33.97, 66.46], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [9]:
density = ee.Kernel.square(radius=2)

imgdensity = img.select('SR_B5_p20').neighborhoodToBands(density)
swir_dist = imgdensity.reduce(ee.Reducer.percentile([10, 20, 30, 40, 50, 60, 70, 80, 90]))

ndvi = img.normalizedDifference(['SR_B3_p20', 'SR_B4_p20'])
ndvidensity = ndvi.neighborhoodToBands(density)
ndvi_dist = ndvidensity.reduce(ee.Reducer.percentile([10, 20, 30, 40, 50, 60, 70, 80, 90]))

stacked = ee.Image([img, imageTier1, imageTier2, imageTier3, imageTier4, imageTier5, imageTier6, swir_dist, ndvi_dist])
stacked

In [10]:
map = geemap.Map(center=[33.97, 66.46], zoom=5)
map.addLayer(stacked, {
    'min': 0,
    'max': 1,
    'bands': ['SR_B3_p20', 'SR_B2_p20', 'SR_B1_p20']
}, "Stacked Images")

In [11]:
train_shp = './afg/afg_train.shp'
train = geemap.shp_to_ee(train_shp)

stacked = stacked.select(['SR_B2_p20', 'SR_B3_p20', 'SR_B4_p20', 'SR_B5_p20', 'SR_B2_p20_1', 'SR_B3_p20_1', 'SR_B4_p20_1', 'SR_B5_p20_1', 'SR_B2_p20_2', 'SR_B3_p20_2', 'SR_B4_p20_2', 'SR_B5_p20_2'])
example_pnts = stacked.sampleRegions(
    collection=train, scale=10, geometries=True
)

sample = example_pnts.randomColumn()
training = sample.filter(ee.Filter.lt('random', 0.8))
testing = sample.filter(ee.Filter.gte('random', 0.8))

classifier = ee.Classifier.smileRandomForest(numberOfTrees=200).train(
    features=training,
    classProperty='lc_id',
    inputProperties=img.bandNames()
)
img_class = stacked.focal_mode(2)
img_class = img_class.classify(classifier)

In [12]:
Map = geemap.Map()

visClass = {
    'min' : 0,
    'max': 6,
    'palette': ['white', 'green', 'red', 'blue', 'yellow', 'purple', 'grey']
}

Map.addLayer(img_class, visClass, "Classified")
Map.centerObject(aoi, 5)
Map

EEException: Computation timed out.